<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/bysykkel_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bysyykel Oslo

## Uploading packages and data

In [1]:
#Importing necessary packages
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Remove previous versions of the uploaded excel file
!rm 2021_04_bysykkel.csv

In [2]:
#Uploading file from local drive
from google.colab import files
uploaded = files.upload()

Saving 2021_04_bysykkel.csv to 2021_04_bysykkel.csv


In [4]:
#Storing dataset in a Pandas Dataframe
import io
by_df = pd.read_csv(io.BytesIO(uploaded['2021_04_bysykkel.csv']), parse_dates=True, infer_datetime_format=True)

In [5]:
#Checking the dataframe information
by_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116864 entries, 0 to 116863
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   started_at                 116864 non-null  object 
 1   ended_at                   116864 non-null  object 
 2   duration                   116864 non-null  int64  
 3   start_station_id           116864 non-null  int64  
 4   start_station_name         116864 non-null  object 
 5   start_station_description  116858 non-null  object 
 6   start_station_latitude     116864 non-null  float64
 7   start_station_longitude    116864 non-null  float64
 8   end_station_id             116864 non-null  int64  
 9   end_station_name           116864 non-null  object 
 10  end_station_description    116858 non-null  object 
 11  end_station_latitude       116864 non-null  float64
 12  end_station_longitude      116864 non-null  float64
dtypes: float64(4), int64(3), obje

##Data cleaning

In [14]:
#Convert columns into datetime object
by_df['started_at'] = pd.to_datetime(by_df['started_at'], format='%Y-%m-%d %H:%M:%S')
by_df['ended_at'] = pd.to_datetime(by_df['ended_at'], format='%Y-%m-%d %H:%M:%S')
by_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116864 entries, 0 to 116863
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   started_at                 116864 non-null  datetime64[ns, UTC]
 1   ended_at                   116864 non-null  datetime64[ns, UTC]
 2   duration                   116864 non-null  int64              
 3   start_station_id           116864 non-null  int64              
 4   start_station_name         116864 non-null  object             
 5   start_station_description  116858 non-null  object             
 6   start_station_latitude     116864 non-null  float64            
 7   start_station_longitude    116864 non-null  float64            
 8   end_station_id             116864 non-null  int64              
 9   end_station_name           116864 non-null  object             
 10  end_station_description    116858 non-null  object      

In [15]:
#Checking the dataframe head
by_df.head(2)

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,2021-04-01 03:14:29.999000+00:00,2021-04-01 03:24:46.822000+00:00,616,407,Sagene bussholdeplass,langs Kierschovs gate,59.937743,10.751648,527,Biskop Gunnerus' gate,ved Oslo City,59.912334,10.752292
1,2021-04-01 03:38:10.860000+00:00,2021-04-01 03:49:05.972000+00:00,655,744,Hallénparken,ved Vogts gate,59.931530,10.762169,522,Mandalls gate,ved Grønlandsleiret,59.912347,10.763815


In [13]:
#Performing count of stations
by_df['end_station_id'].value_counts()

494     2007
479     1645
551     1577
398     1571
421     1543
        ... 
616       60
601       53
742       50
498       43
1919      28
Name: end_station_id, Length: 250, dtype: int64